<center> <h1>PREDICIÓN DEL SUEÑO CON SVM</h1> </center>
<center> <h4>Noemi González, Roberto Holgado y Carmen Plaza seco</h4> </center>


###### Indice

En este jupyter notebook se lleva a cabo la construcción de un clasificador SVM tanto de salida binaria como multiclase para predecir tanto el estado de vigilia de los pacientes como el estado de sueño y sus distintas fases.

* **1. Cargar datos Train-Test**
  * **1.1 Preparación datos de entrada y salida**
* **2. Clasificador SVM Binario**
   * **2.1 Creación del modelo y selección parámetros libres**
   * **2.2 Búsqueda de parámetros con validación cruzada.**
   * **2.3 Evaluación del modelo**
* **3. Clasficador SVM Multiclase**
   * **3.1 Creación del modelo y selección parámetros libres**
   * **3.2 Búsqueda de parámetros con validación cruzada.**
   * **3.3 Evaluación del modelo**
   * **3.4 Visualización de residuos** 

IMPORTAR MÓDULOS

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import statistics as stats
from sklearn.model_selection import GridSearchCV
from random import sample
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve

## 1. CARGAR DATOS TRAIN-TEST

Se cargan los datos de train y test generados en los jupyter notebooks 'Nombre de los jupyter', en los que procesan los datos y se obtienen las características que se utilizan para entrenar los clasificadores. 

En el siguiente apartado se preparan los datos para seleccionar una salida binaria o multiclase según el clasificador que se vaya a utilizar.

### 2.1 Preparación datos de entrada y salida

In [2]:
#Train
Train= pd.read_csv('Train_balanced.csv')
#.drop(['Unnamed: 0'], axis = 1)
Test= pd.read_csv('Test.csv')
#.drop(['Unnamed: 0'], axis = 1)

In [3]:
X_train=Train.drop(['Etiquetas multiclase', 'Etiquetas binarias','ID'], axis = 1) 
y_train_bin=Train['Etiquetas binarias']
y_train_multi=Train['Etiquetas multiclase']

X_test=Test.drop(['Etiquetas multiclase', 'Etiquetas binarias','ID'], axis = 1)
y_test_bin=Test['Etiquetas binarias']
y_test_multi=Test['Etiquetas multiclase']

In [4]:
round(y_test_bin.describe(),2)

count    700.00
mean       0.66
std        0.47
min        0.00
25%        0.00
50%        1.00
75%        1.00
max        1.00
Name: Etiquetas binarias, dtype: float64

In [5]:
X_train.describe()

,Min_hr,Max_hr,Mean_hr,Men_acc_sqrt,Min_acc_x,Max_acc_x,Mean_acc_x,Min_acc_y,Max_acc_y,Mean_acc_y,Min_acc_z,Max_acc_z,Mean_acc_z
count,770.000000,770.000000,770.000000,770.000000,770.000000,770.000000,770.000000,770.000000,770.000000,770.000000,770.000000,770.00000,770.000000
mean,64.748052,69.123377,66.792714,0.997948,-0.163273,-0.073143,-0.118091,-0.132013,0.005532,-0.066065,-0.499403,-0.38526,-0.448584
std,10.603942,12.151690,10.988692,0.005295,0.386774,0.406839,0.369559,0.513189,0.558538,0.498915,0.625732,0.63801,0.614425
min,43.000000,43.000000,43.000000,0.990000,-1.180000,-0.940000,-0.950000,-1.500000,-0.990000,-0.990000,-2.800000,-1.00000,-1.000000
25%,58.000000,60.250000,59.000000,0.990000,-0.430000,-0.360000,-0.390000,-0.540000,-0.430000,-0.450000,-0.930000,-0.88000,-0.890000
50%,65.000000,68.000000,66.500000,1.000000,-0.250000,-0.190000,-0.230000,-0.130000,0.005000,-0.035000,-0.815000,-0.74000,-0.770000
75%,71.750000,77.000000,73.777500,1.000000,0.080000,0.220000,0.140000,0.217500,0.330000,0.267500,-0.190000,0.10750,-0.052500
max,103.000000,111.000000,106.570000,1.030000,0.830000,1.450000,0.830000,0.980000,3.620000,0.990000,0.990000,1.61000,1.000000


## 2. CLASIFICADOR SVM BINARIO

En este primer clasificador, se ha utilizado el método SVM para clasificar la vigilia o sueño ligero del sueño profundo. Es decir, se ha construido un clasificador SVM con salida binaria.

### 2.1  Creación del modelo y selección parámetros libres

En primer lugar, se construye un clasificador SVM de salida binaria sin búsqueda de parámetros. Se fija el valor de gamma o el valor de C y el otro se estima mediante la búsqueda del parámetro óptimo.

En el siguiente código se muestra como fijar un parámetro y barrer los posibles valores del otro.

In [6]:
clf = SVC(kernel='linear', gamma=1,decision_function_shape='ovr' ) # gamma=1 o C=10, depende del que queramos fijar
modelo_base=clf.fit(X_train,y_train_bin)

In [7]:
clf

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

### 2.2  Búsqueda de parámetros con validación cruzada

A continuación, se realiza la búsqueda de parámetros. El parámetro gamma o C se fija y se hace un 'barrido' de los posibles valores del parámetro no fijado para elegir aquel que maximice la exactitud (accuracy).

Tras varias iteraciones se ha visto que los mejores parámetros son gamma=1 y C=10 para el clasificadro binario.


In [ ]:
Cs =[0.1,1,10,100,1000]
# gamma = range(1,10,1) #si se quisiese estimar gamma
param_grid = { 'C': Cs  } #'gamma': gamma
svc_grid = GridSearchCV(clf, param_grid, scoring='accuracy',cv=5,n_jobs=-1)
svc_grid.fit(X_train, y_train_bin)
best_param=svc_grid.best_params_

In [ ]:
# Clasificador con los mejores parámetros
svc_clf=svc_grid.best_estimator_

In [ ]:
#parametro que queremos estimar
"""
train_scores, test_scores = validation_curve(svc_clf, X_train, y_train_bin, param_name="C", param_range=Cs,cv=5)
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
plt.title("Validation Curve with SVM ")
plt.xlabel('C')
plt.ylabel("Accuracy")

plt.plot(Cs, train_scores_mean, label="Training score",color="navy",marker='.')
plt.plot(Cs, test_scores_mean, label="Cross-validation score",color="darkorange",marker='.')
plt.grid()
plt.legend(loc="best")
plt.show()
"""

In [ ]:
print("El valor más óptimo para el parámetro a estimar es:",best_param)

In [ ]:
print("Tasa de acierto del modelo base:",modelo_base.score(X_test,y_test_bin))
print("Tasa de acierto con la busqueda de parámetros:",svc_clf.score(X_test,y_test_bin))

#Mejora
print('Mejora  del {:0.2f}%.'.format( 100 * (svc_clf.score(X_test,y_test_bin) - modelo_base.score(X_test,y_test_bin)) / modelo_base.score(X_test,y_test_bin)))

### 2.3 Evaluación del modelo

In [ ]:
predict = svc_clf.predict(X_test)

#EVALUACION
acc_test= svc_clf.score(X_test, y_test_bin).round(4)
acc_train= svc_clf.score(X_train, y_train_bin).round(4)
print('La exactitud para el modelo de SVM en el conjunto de TEST es:',acc_test,'\n')
print('La exactitud para el modelo de SVM en el conjunto de TRAIN es:',acc_train,'\n')

print('Confusion Matrix')
print(confusion_matrix(y_test_bin, predict),'\n')

print("=============== Classification Report ================")
print(classification_report(y_test_bin, predict))
#matriz_confusion=pd.crosstab(Y_test,predict, rownames=['Clase predicha'], colnames=['Resultado del clasificador'])

In [ ]:
matriz_confusion=pd.crosstab(y_test_bin,predict, rownames=['Clase predicha'], colnames=['Resultado del clasificador'])

In [ ]:
matriz_confusion

### 2.4 Visualización de resultados

In [ ]:
#EJEMPLO LOS 10 PRIMEROS
pacientes = len(y_test_bin[0:30])
t = np.arange(0,pacientes)
plt.figure(figsize=(15,9))
plt.plot(t,y_test_bin[0:30],color='blue', marker='.',label='Real')
plt.plot(t,predict[0:30],color='green', marker='.',label='Predicha')

plt.title('Fase del sueño real vs. predicha en los primeros 30s')
plt.xlabel('Muestras de pacientes')
plt.ylabel('Fase del sueño')
plt.yticks([0,1])
#plt.xlim([0,10])
plt.legend()

## 3. CLASIFICADOR SVM MULTICLASE

Este segundo clasificador SVM tiene una salida multiclase y clasifica tanto el estado de vigilia como las distintas etapas de sueño ligero y profundo.

### 3.1 Creación del modelo y selección parámetros libres

En primer lugar, se construye un clasificador SVM de salida multiclase sin búsqueda de parámetros. Se fija el valor de gamma y se deja el valor de C por defecto.

In [ ]:
clf = SVC(kernel='linear',gamma=0.1,decision_function_shape='ovr')
modelo_base=clf.fit(X_train,y_train_multi)

### 3.2 Búsqueda de parámetros con validación cruzada

In [ ]:
Cs =[0.1,1,10,100,1000]
param_grid = {'C': Cs}
svc_grid = GridSearchCV(clf, param_grid, scoring='accuracy',cv=5,n_jobs=-1)
svc_grid.fit(X_train, y_train_multi)
C=svc_grid.best_params_

In [ ]:
# Clasificador con los mejores parámetros
svc_clf=svc_grid.best_estimator_

In [ ]:
#parametro C
"""
train_scores, test_scores = validation_curve(svc_clf, X_train, y_train_multi, param_name="C", param_range=Cs,cv=5)
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
plt.title("Validation Curve with SVM ")
plt.xlabel('C')
plt.ylabel("Accuracy")

plt.plot(Cs, train_scores_mean, label="Training score",color="navy",marker='.')
plt.plot(Cs, test_scores_mean, label="Cross-validation score",color="darkorange",marker='.')
plt.grid()
plt.legend(loc="best")
plt.show()
"""

In [ ]:
print("C que maximiza la accuracy:",C)

In [ ]:
print("Tasa de acierto del modelo base:",modelo_base.score(X_test,y_test_multi))
print("Tasa de acierto con la busqueda de parámetros:",svc_clf.score(X_test,y_test_multi))

#Mejora
print('Mejora  del {:0.2f}%.'.format( 100 * (svc_clf.score(X_test,y_test_multi) - modelo_base.score(X_test,y_test_multi)) / modelo_base.score(X_test,y_test_multi)))

### 3.3 Evaluación del modelo

In [ ]:
predict = svc_clf.predict(X_test)

#EVALUACION
acc_test= svc_clf.score(X_test, y_test_multi).round(4)
acc_train= svc_clf.score(X_train, y_train_multi).round(4)
print('La exactitud para el modelo de SVM en el conjunto de TEST es:',acc_test,'\n')
print('La exactitud para el modelo de SVM en el conjunto de TRAIN es:',acc_train,'\n')

print('Confusion Matrix')
print(confusion_matrix(y_test_multi, predict),'\n')

print("=============== Classification Report ================")
print(classification_report(y_test_multi, predict))

In [ ]:
matriz_confusion=pd.crosstab(y_test_multi,predict, rownames=['Clase predicha'], colnames=['Resultado del clasificador'])

In [ ]:
matriz_confusion

### 3.4 Visualización predicción del sueño

In [ ]:
pacientes = len(y_test_multi)
t = np.arange(0,pacientes)
plt.figure(figsize=(15,9))
plt.plot(t,y_test_multi,color='blue', marker='.',label='Real')
plt.plot(t,predict,color='green', marker='.',label='Predicha')

plt.title('Fase del sueño real vs. predicha en los primeros 30s')
plt.xlabel('Muestras de pacientes')
plt.ylabel('Fase del sueño')
plt.legend()

* Estimamos parámetro por parámetro porque hacer los dos a la vez tarda mucho
* No se por qué pero al poner kernel='rbf' se hace todo rapidisimo pero solo reconoce una clase (no lo hace nada bien pero va rapido).
* No se detectan todas las clases en el multiclase --> necesitamos entrenar con mas datos y luego probar a estimar un parámetro y el otro como en el binario
* Baja accuracy en multiclase